Notebook to load, create, and prepare the data

In [ ]:
#import root_pandas
import pandas as pd
import numpy as np
from numpy import random
import copy
from sklearn.preprocessing import StandardScaler
from pickle import dump
from pickle import load

Create a function to process a list of given DataFrames

In [ ]:
custom_filename = '../RECO/RECO_out_'
custom_ending = '_pi0_df.root'
# path = custom_filename + number of run + custom_ending, e.g. '../RECO/RECO_out_1_pi0_df.root'
num_of_runs = 4

def shuffle_df(data):
    # Shuffes a given DataFrame

    indices = np.arange(data.values.shape[0])
    np.random.shuffle(indices)

    return pd.DataFrame(data.values[indices], columns=data.columns)

def process_data(data_temp, i=-1):
    data_temp = cut_pi0s(data_temp)
    data_temp = create_dataset(data_temp)
    data_temp = normalize_data(data_temp)
    data_temp['run'] = i
    
    return data_temp

print('-'*30, '\nDATASET 1\n')
#data = process_data(root_pandas.read_root('/media/sf_shared_folder/RECO/RECO_out_1_pi0.root'), 1)
data = process_data(pd.read_pickle(custom_filename + '1' + custom_ending), 1)

for i in np.arange(2,(num_of_runs+1)):
    print('\n', '-'*30, '\nDATASET ', i, '\n', sep='')    
    
    path = custom_filename + str(i) + custom_ending
    #pi0s = root_pandas.read_root(path)
    pi0s = pd.read_pickle(path)
    
    data = pd.concat([data, process_data(pi0s, i)])

data = shuffle_df(data)
data.to_pickle('dataset.pkl')

print('\n', '-'*30, '\nDone\n', sep='')
print(sum(data.y), len(data.y))

data.head(20)

Define some helper methods

In [ ]:
def get_wanted_gammas(gammas_temp, p_cut=0.13):
    # Get a DataFrame with all wanted gammas
    return copy.deepcopy(gammas_temp.loc[(abs(gammas_temp.genMotherPDG__bo1__bc)==423) & (abs(gammas_temp.genMotherPDG)==111) & (gammas_temp.isSignal==1) & (gammas_temp.p < p_cut)])

def get_wanted_pi0s(pi0s_temp, p_cut=0.23):
    # Get a DataFrame with all wanted pi0s
    return copy.deepcopy(pi0s_temp.loc[(abs(pi0s_temp.genMotherPDG)==423) & (pi0s_temp.isSignal==1) & (pi0s_temp.p < p_cut)])

def compute_cut_efficiency(before_cut_data, after_cut_data, is_pi0s=False):
    # Compute the efficiency of a cut
    
    if is_pi0s:
        result = np.array([(len(after_cut_data)/len(before_cut_data)), (len(get_wanted_pi0s(after_cut_data))/len(get_wanted_pi0s(before_cut_data))), len(get_wanted_pi0s(after_cut_data)), len(after_cut_data)])
    else:
        result = np.array([(len(after_cut_data)/len(before_cut_data)), (len(get_wanted_gammas(after_cut_data))/len(get_wanted_gammas(before_cut_data))), len(get_wanted_gammas(after_cut_data)), len(after_cut_data)])

    # The array contains the following information:
    # [% of remaining dataset, % of remaining wanted data, # of remaining wanted datapoints, # of remaining datapoints]
    print(result[0], result[1], '\n', result[2], result[3], '\n') # TODO format output
    return result

#NOTE: Note: PDG: 111: pi0 ; 423: D*(2007)

Cut the $\pi^0$ list. Keep only $\pi^0$'s where dM is in the range -0.075 and 0.02 and where the momentum is less than 0.23 GeV/c

In [ ]:
def cut_pi0s(pi0s_temp):
    pi0s_cutted_temp = copy.deepcopy(pi0s_temp.loc[abs(pi0s_temp.dM > -75E-3) & abs(pi0s_temp.dM < 2E-2) & (pi0s_temp.p > 0.23)])
    compute_cut_efficiency(pi0s_temp, pi0s_cutted_temp, is_pi0s=True)
    return pi0s_cutted_temp

Create a dataframe containing all $\gamma$ combinations based on the $\pi^0$ list. The column names of the first $\gamma$ have '_2' appended and the first gamma '_1'.

Note: All combinations are only created in one permutation

In [ ]:
def get_y(genMotherPDG, isSignal):
    return int((abs(genMotherPDG) == 423) & (isSignal == 1))

def simple_drop(y, drop_probability):
    return ((y==1) | (random.uniform(0, 1) > drop_probability))

def create_dataset(pi0s, drop_probability=0.95):

    data_temp = copy.deepcopy(pi0s)

    #data_temp['y'] = data_temp.apply(lambda x: int((abs(x.genMotherPDG) == 423) & (x.isSignal == 1)), axis=1)
    vfunc = np.vectorize(get_y)
    data_temp['y'] = vfunc(data_temp.genMotherPDG, data_temp.isSignal)
    
    #data_temp['flag'] = data_temp.apply(lambda x: ( (x.y==1) | (random.uniform(0, 1) > drop_probability)), axis=1)
    vfunc = np.vectorize(simple_drop)
    data_temp['flag'] = vfunc(data_temp.y, drop_probability)
    
    data_temp = data_temp.loc[data_temp.flag==1].drop('flag', axis=1)

    data_temp.drop(['__experiment__', '__run__', '__event__', '__candidate__', '__ncandidates__', '__weight__', 'genMotherID', 
                 'genMotherPDG', 'genMotherPDG__bo1__bc', 'genMotherPDG__bo2__bc', 'isSignal', 'mdstSource',
                   'daughter__bo0__cm__spmdstSource__bc', 'daughter__bo1__cm__spmdstSource__bc', 'p', 'dM']
                   , axis=1, inplace=True, errors='ignore')

    # Print some info about the created dataset
    print('Size of the dataset: (x_data, y_data)', len(data_temp))
    print('Number of real pi0 combinations:', sum(data_temp['y']))
    
    return data_temp

Normalize the data that all values have a mean of 0 and a standard deviation of 1.

In [ ]:
scaler = load(open('scaler.pkl', 'rb'))

def normalize_data(data_temp, scale_localy=False):
    y_values = data_temp.y
    data_temp = data_temp.drop(['y'], axis=1)
    
    if scale_localy:
        data_temp[data_temp.columns] = scaler.fit_transform(data_temp[data_temp.columns])
    else:
        data_temp[data_temp.columns] = scaler.transform(data_temp[data_temp.columns])
        
    data_temp['y'] = y_values
    
    return data_temp


-----------------


Create a list where the amount of wanted and unwanted combinations is balanced and save it in files. The dataset is balanced by simply dropping some unwanted combinations.

In [ ]:
def drop_data(data_temp, drop_propability):
    # Function that drops an unwanted combination with a given probability
    
    data_temp = copy.deepcopy(data_temp)

    data_temp['flag'] = data_temp.apply(lambda x: ( (x.y==1) | (random.uniform(0, 1) > drop_propability)), axis=1)
    result = data_temp.loc[data_temp.flag==1].drop('flag', axis=1)
    
    print(sum(result.y), len(result.y))    
    return result

Create a StandardScaler with the 10 first dataset and save it

In [ ]:
def shorten_data(data_temp):
    data_temp = cut_pi0s(data_temp)
    data_temp.drop(['__experiment__', '__run__', '__event__', '__candidate__', '__ncandidates__', '__weight__', 'genMotherID', 
                 'genMotherPDG', 'genMotherPDG__bo1__bc', 'genMotherPDG__bo2__bc', 'isSignal', 'mdstSource',
                   'daughter__bo0__cm__spmdstSource__bc', 'daughter__bo1__cm__spmdstSource__bc', 'p', 'dM']
                , axis=1, inplace=True, errors='ignore')
    print(len(data_temp.columns))
    
    return data_temp

print('-'*30, '\nDATASET 1\n')
data = shorten_data(pd.read_pickle('../RECO/RECO_out_1_pi0_df.root'))

for i in np.arange(2,51):
    print('\n', '-'*30, '\nDATASET ', i, '\n', sep='')    
    
    path = '../RECO/RECO_out_' + str(i) + '_pi0_df.root'
    data = pd.concat([data, shorten_data(pd.read_pickle(path))])
    print(data.shape)
    
    
scaler = StandardScaler().fit(data[data.columns])

dump(scaler, open('scaler.pkl', 'wb'))

del data, scaler

Remove collinear columns. To remove collinear columns from the data I am using VIF (https://www.analyticsvidhya.com/blog/2020/03/what-is-multicollinearity/) with a threshold of 10.

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=10.0):
    # https://stats.stackexchange.com/questions/155028/how-to-systematically-remove-collinear-variables-in-python

    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

def remove_collinear_rows(gammas_temp):
    # Removes collinear columns but ignores the mdstSource column
    gammas_temp = copy.deepcopy(gammas_temp)
    mdstSource_temp = gammas_temp.mdstSource
    gammas_temp.drop(['__experiment__', '__run__', '__event__', '__candidate__', '__ncandidates__', '__weight__', 'genMotherID',
                 'genMotherP','M', 'dM', 'InvM', 'E', 'E_uncertainty', 'clusterE9E21', 'b2bClusterTheta', 'b2bClusterPhi',
                 'clusterPhi', 'clusterErrorPhi', 'clusterTheta', 'clusterE', 'clusterErrorE', 'ECLClusterE_uncertainty', 
                 'genMotherPDG', 'genMotherPDG__bo1__bc', 'genMotherPDG__bo2__bc', 'isSignal', 'mdstSource']
                , axis=1, inplace=True, errors='ignore')
    
    return pd.concat([mdstSource_temp, calculate_vif_(X=gammas_temp)], axis=1, join='inner')